In [30]:
import pandas as pd

korean_df = pd.read_csv('datasets/korean_sample.csv')
madalena_df = pd.read_csv('datasets/madalena_sample.csv')

korean_df["timestamp"] = pd.to_datetime(korean_df["timestamp"])
madalena_df["timestamp"] = pd.to_datetime(madalena_df["timestamp"])

if "Unnamed: 0" in madalena_df.columns:
    madalena_df.drop(columns=["Unnamed: 0"], inplace=True)

korean_df["house_id"] = korean_df["house_id"].astype(str)
madalena_df["house_id"] = madalena_df["house_id"].astype(str)

madalena_start = madalena_df["timestamp"].min()
korean_start = korean_df["timestamp"].min()
initial_shift = korean_start - madalena_start
madalena_df["timestamp"] = madalena_df["timestamp"] + initial_shift

common_start = max(korean_df["timestamp"].min(), madalena_df["timestamp"].min())
common_end = min(korean_df["timestamp"].max(), madalena_df["timestamp"].max())
korean_df = korean_df[(korean_df["timestamp"] >= common_start) & (korean_df["timestamp"] <= common_end)]
madalena_df = madalena_df[(madalena_df["timestamp"] >= common_start) & (madalena_df["timestamp"] <= common_end)]
shift_offset = korean_df["timestamp"].min() - madalena_df["timestamp"].min()

madalena_df["timestamp"] = madalena_df["timestamp"] - shift_offset
madalena_df = madalena_df.drop_duplicates(subset=["timestamp", "house_id"])

madalena_df.set_index("timestamp", inplace=True)

madalena_minutely = madalena_df.groupby("house_id").resample("1T").ffill().reset_index(level=0, drop=True).reset_index()


# .groupby("house_id").resample("1T").ffill().reset_index(level=0, drop=True).reset_index()

madalena_minutely.head()


C:\Users\Kelsier\AppData\Local\Temp\ipykernel_9356\1228422609.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  madalena_minutely = madalena_df.groupby("house_id").resample("1T").ffill().reset_index(level=0, drop=True).reset_index()


,timestamp,total_energy,CO2[ppm],PM4[ug/m3],Lighting[lux],T_in[°C],RH [%],PM10[ug/m3],PM2_5[ug/m3],PM1[ug/m3],PM0_5[ug/m3],T_out [°C],house_id
0,2016-09-01 00:00:00,71.77,624.23,65.43,0.0,28.46,55.51,65.44,65.42,65.22,56.81,24.5,145
1,2016-09-01 00:01:00,71.77,624.23,65.43,0.0,28.46,55.51,65.44,65.42,65.22,56.81,24.5,145
2,2016-09-01 00:02:00,71.77,624.23,65.43,0.0,28.46,55.51,65.44,65.42,65.22,56.81,24.5,145
3,2016-09-01 00:03:00,71.77,624.23,65.43,0.0,28.46,55.51,65.44,65.42,65.22,56.81,24.5,145
4,2016-09-01 00:04:00,71.77,624.23,65.43,0.0,28.46,55.51,65.44,65.42,65.22,56.81,24.5,145


In [32]:
merged_df = korean_df.merge(madalena_minutely, on=["timestamp", "house_id"], how="left")
merged_df.head()

,timestamp,total_active_power,total_reactive_power,washing-machine_active_power,washing-machine_reactive_power,rice-cooker_active_power,rice-cooker_reactive_power,water-purifier_active_power,water-purifier_reactive_power,microwave_active_power,...,CO2[ppm],PM4[ug/m3],Lighting[lux],T_in[°C],RH [%],PM10[ug/m3],PM2_5[ug/m3],PM1[ug/m3],PM0_5[ug/m3],T_out [°C]
0,2016-11-01 00:00:00,226.278702,-41.480791,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-11-01 00:01:00,426.839889,-85.463171,0.117980,6.110303,0.000000,0.000000,340.030000,-1.647143,0.475614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-11-01 00:02:00,417.073146,-142.700712,0.055135,5.976013,0.253236,0.018027,338.550340,-1.636741,0.496926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-11-01 00:03:00,147.948077,-145.353986,0.034107,6.011007,0.218430,0.029089,68.882708,-0.310016,0.512941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-11-01 00:04:00,68.593354,-145.734066,0.045174,6.006146,0.264247,0.036712,2.012237,0.099879,0.517540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
